In [26]:
import os
from typing import List
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

In [27]:
# 列出资源目录下的图书信息 json 文件路径列表
fpaths = [f'../resources/{fname}' for fname in os.listdir('../resources/') 
          if fname.endswith('.json') and '_edited' not in fname]
fpaths

['../resources/经济学.json',
 '../resources/计算机科学.json',
 '../resources/中国文学.json',
 '../resources/金融学.json',
 '../resources/哲学.json']

### loading book documents

In [28]:
# 合并文本字段作为 document content
def combine_book_fields(fpath: str) -> List[Document]:
    # 加载 json 文档，查看数据样式
    books_computer = json.loads(Path(fpath).read_text())
    
    # 数据处理，合并图书各项信息作为 document content
    for book in books_computer:
        book['document_content'] = (
        f'书名：《{book["book_name"]}》\n'
        f'作者：{book["author"]}\n'
        f'分类：{book["classify"]}\n'
        f'价格：{book["price"]}\n'
        f'发布年份：{book["publish_date"].split("/")[-1]}\n'
        f'出版社：{book["press"]}\n'
        f'简介：{book["summary"]}')

    edited_file_path = f"{fpath.split('.json')[0]}_edited.json"
    # 重新存入文件
    with open(edited_file_path, 'w') as f:
        f.write(json.dumps(books_computer))
    
    # 文档 metadata 函数，用于关联对应的 Document
    def metadata_func(record: dict, metadata: dict) -> dict:
        metadata.update(record)
        if 'document_content' in metadata:
            del metadata['document_content']
        if 'source' in metadata:
            metadata['source'] = metadata['source'].split('/')[-1]
        return metadata
    
    # 使用 JSONLoader 加载json文档
    # 使用 JSONLoader 加载json文档
    loader = JSONLoader(edited_file_path, 
                        jq_schema='.[]',
                        content_key='document_content',
                        metadata_func=metadata_func)
    return loader.load()

In [36]:
all_document = [combine_book_fields(fpath) for fpath in fpaths]

In [43]:
documents = [doc for documents in all_document for doc in documents]
print(len(documents))

50


In [44]:
# initialize openai embedding
embedding = OpenAIEmbeddings()

In [45]:
db = FAISS.from_documents(documents, embedding)

In [46]:
db.save_local('books')

In [49]:
db.similarity_search('编程', k=3)

[Document(page_content='书名：《Python 编程入门》\n作者：Eric Matthes\n分类：编程语言\n价格：59.99\n发布年份：2020-03\n出版社：No Starch Press\n简介：这本书是初学者学习 Python 编程的绝佳选择，包含大量示例和练习，逐步引导读者掌握 Python 编程基础。', metadata={'source': '计算机科学_edited.json', 'seq_num': 1, 'book_name': 'Python 编程入门', 'author': 'Eric Matthes', 'classify': '编程语言', 'price': 59.99, 'publish_date': '2020-03', 'press': 'No Starch Press', 'summary': '这本书是初学者学习 Python 编程的绝佳选择，包含大量示例和练习，逐步引导读者掌握 Python 编程基础。'}),
 Document(page_content="书名：《Web开发权威指南》\n作者：Eric Freeman, Elisabeth Robson, Kathy Sierra, Bert Bates\n分类：Web开发\n价格：69.99\n发布年份：2006-11\n出版社：O'Reilly Media\n简介：这本书全面介绍了 Web 开发的各个方面，包括前端和后端开发技术。", metadata={'source': '计算机科学_edited.json', 'seq_num': 7, 'book_name': 'Web开发权威指南', 'author': 'Eric Freeman, Elisabeth Robson, Kathy Sierra, Bert Bates', 'classify': 'Web开发', 'price': 69.99, 'publish_date': '2006-11', 'press': "O'Reilly Media", 'summary': '这本书全面介绍了 Web 开发的各个方面，包括前端和后端开发技术。'}),
 Document(page_content='书名：《人机交互: 设计原理与实践》\n作者：Alan Cooper, Robert

In [50]:
db.similarity_search('我想买本哲学方面的书', k=3)

[Document(page_content='书名：《西方哲学史》\n作者：Bertrand Russell\n分类：哲学史\n价格：49.99\n发布年份：2018-04\n出版社：Routledge\n简介：这本书回顾了西方哲学的历史发展，从古希腊哲学到现代哲学思想，涵盖了众多哲学家和哲学流派。', metadata={'source': '哲学_edited.json', 'seq_num': 1, 'book_name': '西方哲学史', 'author': 'Bertrand Russell', 'classify': '哲学史', 'price': 49.99, 'publish_date': '2018-04', 'press': 'Routledge', 'summary': '这本书回顾了西方哲学的历史发展，从古希腊哲学到现代哲学思想，涵盖了众多哲学家和哲学流派。'}),
 Document(page_content='书名：《经济学的思维方式》\n作者：Paul Heyne\n分类：经济学方法论\n价格：49.99\n发布年份：2015-12\n出版社：Liberty Fund\n简介：这本书讨论了经济学的思考方式，帮助读者理解和应用经济学原理来解决问题。', metadata={'source': '经济学_edited.json', 'seq_num': 6, 'book_name': '经济学的思维方式', 'author': 'Paul Heyne', 'classify': '经济学方法论', 'price': 49.99, 'publish_date': '2015-12', 'press': 'Liberty Fund', 'summary': '这本书讨论了经济学的思考方式，帮助读者理解和应用经济学原理来解决问题。'}),
 Document(page_content='书名：《知识与信仰》\n作者：Alvin Plantinga\n分类：宗教哲学\n价格：31.99\n发布年份：2015-11\n出版社：Oxford University Press\n简介：这本书探讨了知识、信仰和神的存在之间的关系，是神学和哲学的交叉领域研究。', metadata={'source': '哲学_edited.json', 'seq_nu

In [52]:
db.similarity_search('你这里有发布年份为2015年的书吗', k=3)

[Document(page_content='书名：《活着》\n作者：余华\n分类：现代小说\n价格：29.99\n发布年份：1993-05\n出版社：上海文艺出版社\n简介：这是一部描写中国社会动荡历史下一个人命运的小说，深刻反映了生存的苦难和人性的坚韧。', metadata={'source': '中国文学_edited.json', 'seq_num': 2, 'book_name': '活着', 'author': '余华', 'classify': '现代小说', 'price': 29.99, 'publish_date': '1993-05', 'press': '上海文艺出版社', 'summary': '这是一部描写中国社会动荡历史下一个人命运的小说，深刻反映了生存的苦难和人性的坚韧。'}),
 Document(page_content='书名：《大数据时代》\n作者：Viktor Mayer-Schönberger, Kenneth Cukier\n分类：大数据\n价格：49.99\n发布年份：2013-03\n出版社：Houghton Mifflin Harcourt\n简介：这本书探讨了大数据对社会和业务的影响，以及如何在大数据时代中获得洞见。', metadata={'source': '计算机科学_edited.json', 'seq_num': 10, 'book_name': '大数据时代', 'author': 'Viktor Mayer-Schönberger, Kenneth Cukier', 'classify': '大数据', 'price': 49.99, 'publish_date': '2013-03', 'press': 'Houghton Mifflin Harcourt', 'summary': '这本书探讨了大数据对社会和业务的影响，以及如何在大数据时代中获得洞见。'}),
 Document(page_content='书名：《金融数学》\n作者：Martin Baxter, Andrew Rennie\n分类：金融数学\n价格：59.99\n发布年份：2015-06\n出版社：Wiley\n简介：该书介绍了金融数学的基本概念和工具，包括期权定价、风险管理等方面的数学方法。', metadata={'s

In [53]:
db.similarity_search('我想买一本金融方面的书，你这里有吗')

[Document(page_content='书名：《金融市场学》\n作者：William F. Sharpe, Gordon J. Alexander, Jeffrey V. Bailey\n分类：金融学\n价格：79.99\n发布年份：2018-05\n出版社：Pearson\n简介：这本书介绍了金融市场的基本原理和运作方式，涵盖了股票、债券、衍生品等金融工具。', metadata={'source': '金融学_edited.json', 'seq_num': 1, 'book_name': '金融市场学', 'author': 'William F. Sharpe, Gordon J. Alexander, Jeffrey V. Bailey', 'classify': '金融学', 'price': 79.99, 'publish_date': '2018-05', 'press': 'Pearson', 'summary': '这本书介绍了金融市场的基本原理和运作方式，涵盖了股票、债券、衍生品等金融工具。'}),
 Document(page_content='书名：《金融数学》\n作者：Martin Baxter, Andrew Rennie\n分类：金融数学\n价格：59.99\n发布年份：2015-06\n出版社：Wiley\n简介：该书介绍了金融数学的基本概念和工具，包括期权定价、风险管理等方面的数学方法。', metadata={'source': '金融学_edited.json', 'seq_num': 6, 'book_name': '金融数学', 'author': 'Martin Baxter, Andrew Rennie', 'classify': '金融数学', 'price': 59.99, 'publish_date': '2015-06', 'press': 'Wiley', 'summary': '该书介绍了金融数学的基本概念和工具，包括期权定价、风险管理等方面的数学方法。'}),
 Document(page_content='书名：《金融工程学》\n作者：Paul P. Dorfman\n分类：金融工程\n价格：69.99\n发布年份：2019-09\n出版社：Wiley\n简介：该书探讨了金融工程领域的数学、统计

In [54]:
db.similarity_search('你有哪些哲学方面的书')

[Document(page_content='书名：《西方哲学史》\n作者：Bertrand Russell\n分类：哲学史\n价格：49.99\n发布年份：2018-04\n出版社：Routledge\n简介：这本书回顾了西方哲学的历史发展，从古希腊哲学到现代哲学思想，涵盖了众多哲学家和哲学流派。', metadata={'source': '哲学_edited.json', 'seq_num': 1, 'book_name': '西方哲学史', 'author': 'Bertrand Russell', 'classify': '哲学史', 'price': 49.99, 'publish_date': '2018-04', 'press': 'Routledge', 'summary': '这本书回顾了西方哲学的历史发展，从古希腊哲学到现代哲学思想，涵盖了众多哲学家和哲学流派。'}),
 Document(page_content='书名：《知识与信仰》\n作者：Alvin Plantinga\n分类：宗教哲学\n价格：31.99\n发布年份：2015-11\n出版社：Oxford University Press\n简介：这本书探讨了知识、信仰和神的存在之间的关系，是神学和哲学的交叉领域研究。', metadata={'source': '哲学_edited.json', 'seq_num': 7, 'book_name': '知识与信仰', 'author': 'Alvin Plantinga', 'classify': '宗教哲学', 'price': 31.99, 'publish_date': '2015-11', 'press': 'Oxford University Press', 'summary': '这本书探讨了知识、信仰和神的存在之间的关系，是神学和哲学的交叉领域研究。'}),
 Document(page_content='书名：《符号、信仰和神话》\n作者：Mircea Eliade\n分类：宗教哲学\n价格：29.99\n发布年份：1969\n出版社：Harper & Row\n简介：这本书研究了宗教和神话在人类文化中的重要性，以及它们对信仰和意义的影响。', metadata={'source': '哲学_edited.json

In [63]:
db.similarity_search_with_relevance_scores(query='你有Mircea Eliade写的书吗', filter=dict(author="Mircea Eliade"))

[(Document(page_content='书名：《符号、信仰和神话》\n作者：Mircea Eliade\n分类：宗教哲学\n价格：29.99\n发布年份：1969\n出版社：Harper & Row\n简介：这本书研究了宗教和神话在人类文化中的重要性，以及它们对信仰和意义的影响。', metadata={'source': '哲学_edited.json', 'seq_num': 4, 'book_name': '符号、信仰和神话', 'author': 'Mircea Eliade', 'classify': '宗教哲学', 'price': 29.99, 'publish_date': '1969', 'press': 'Harper & Row', 'summary': '这本书研究了宗教和神话在人类文化中的重要性，以及它们对信仰和意义的影响。'}),
  0.7983955656231112)]

In [58]:
db.similarity_search?

Signature:
db.similarity_search(
    query: 'str',
    k: 'int' = 4,
    filter: 'Optional[Dict[str, Any]]' = None,
    fetch_k: 'int' = 20,
    **kwargs: 'Any',
) -> 'List[Document]'
Docstring:
Return docs most similar to query.

Args:
    query: Text to look up documents similar to.
    k: Number of Documents to return. Defaults to 4.
    filter: (Optional[Dict[str, str]]): Filter by metadata. Defaults to None.
    fetch_k: (Optional[int]) Number of Documents to fetch before filtering.
              Defaults to 20.

Returns:
    List of Documents most similar to the query.
File:      ~/Documents/Projects/llm-training-camp/booksales-consultant/.venv/lib/python3.11/site-packages/langchain/vectorstores/faiss.py
Type:      method